In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224, 224]

train_path = 'Dataset'
valid_path = 'Val_Dataset'

In [3]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94773248/94765736 [==============================] - 96s 1us/step


In [4]:
for layer in resnet.layers:
    layer.trainable = False

In [5]:
x = Flatten()(resnet.output)

In [7]:
prediction = Dense(6, activation='softmax')(x)

model = Model(inputs=resnet.input, outputs=prediction)

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=60,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.3,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory('Dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 642 images belonging to 6 classes.


In [14]:
test_set = test_datagen.flow_from_directory('Val_Dataset',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 102 images belonging to 6 classes.


In [15]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
21/21 [==============================] - 160s 8s/step - loss: 6.9012 - accuracy: 0.1978 - val_loss: 4.0287 - val_accuracy: 0.1471
Epoch 2/10
21/21 [==============================] - 115s 5s/step - loss: 3.8674 - accuracy: 0.1978 - val_loss: 3.3879 - val_accuracy: 0.1667
Epoch 3/10
21/21 [==============================] - 128s 6s/step - loss: 2.9794 - accuracy: 0.2134 - val_loss: 3.2227 - val_accuracy: 0.1275
Epoch 4/10
21/21 [==============================] - 125s 6s/step - loss: 2.2397 - accuracy: 0.2773 - val_loss: 2.9970 - val_accuracy: 0.1765
Epoch 5/10
 7/21 [=========>....................] - ETA: 56s - loss: 1.9414 - accuracy: 0.2320

KeyboardInterrupt: 